In [3]:
from ssl import CERT_NONE
from pprint import pprint

from nostr.event import Event
from nostr.key import PrivateKey
from nostr.relay_manager import RelayManager

In [4]:
# Setup Alice's keys
privkey: PrivateKey = PrivateKey().from_nsec(ALICE)
print(privkey.public_key.bech32())

npub1tqh0wyekvvhfc82ux52gd4tahusvfx7wt2789vtvsny3jed5x0gqht6tun


In [5]:
def connect_to_relays(relays:list[str]) -> RelayManager:
    relay_manager = RelayManager()
    for r in relays:
        relay_manager.add_relay(r)
    relay_manager.open_connections({"cert_reqs": CERT_NONE})
    return relay_manager

In [6]:
# Connect to relays
relay_manager = connect_to_relays(
    relays=[
        RELAY,
        "wss://relay.damus.io",
        "wss://relay.snort.social",
        "wss://nos.lol"
    ]
)

pprint(relay_manager.relays)

{'wss://nor.st': <nostr.relay.Relay object at 0x105a44c10>,
 'wss://nos.lol': <nostr.relay.Relay object at 0x105a45150>,
 'wss://relay.damus.io': <nostr.relay.Relay object at 0x105687a90>,
 'wss://relay.snort.social': <nostr.relay.Relay object at 0x105a44f50>}


In [7]:
def create_badge(public_key:str, kind:int=30009, content:str='', tags:list[list[str]]=[],) -> Event:
    return Event(
        public_key=public_key,
        kind=kind,
        content=content,
        tags=tags,
    )

In [8]:
# Create cake card badges
cake_card = create_badge(
    public_key=privkey.public_key.hex(),
    tags = [
        ['d', 'cake-69'],
        ['name', 'Cake Card for $69'],
        ['description', 'Good for $69 of Cake'],
        ['image', 'https://icons.iconarchive.com/icons/iconarchive/fat-sugar-food/256/Cake-icon.png', '1024x1024'],
        ['thumb', 'https://icons.iconarchive.com/icons/iconarchive/fat-sugar-food/256/Cake-icon.png', '256x256']
    ],
)

cake_card_consumed = create_badge(
    public_key=privkey.public_key.hex(),
    tags = [
        ['d', 'poop-69'],
        ['name', 'Consumed Cake Card $69'],
        ['description', '$69 Cake Card consumed, now Poop'],
        ['image', 'https://icons.iconarchive.com/icons/sonya/swarm/256/Poop-icon.png', '1024x1024'],
        ['thumb', 'https://icons.iconarchive.com/icons/sonya/swarm/256/Poop-icon.png', '256x256']
    ],
)

In [9]:
# Sign badge creation events
privkey.sign_event(cake_card)
privkey.sign_event(cake_card_consumed)

print(cake_card.signature)
print(cake_card_consumed.signature)

9b89f818c9c1882f99e0c810d97c814d5536b41e7a75ca0486e69f4c2ddd7de45668c19a0e2863f1f6205c46402aa99a371ea3f9190efab5fa2c15da0a542026
9172dbfed833485463b861fcf27939f97e80b78f4cdb7d5e53a7a37aa385df01a925f722d709b937bccd7fdb3d9c44edef63cd6dfc43693c45e171741a889d34


In [10]:
# Publish badge creation events
relay_manager.publish_event(cake_card)
relay_manager.publish_event(cake_card_consumed)

print(cake_card.id)
print(cake_card_consumed.id)

1806db4b1aa5746b77387dde6f2ee90ef0ce9cc142e888149295573ddeb79248
b5778e284825b908f97f1eba1e9ac381c84bf3c052f0de4e0e008c597eab76e7


# ...
# Lightning Payment Occurs
# ...

In [13]:
def assign_badge(public_key:str, assign_to:str, badge_d:str,
                 kind:int=8, content:str='', tags:list[list[str]]=[]) -> Event:
    return Event(
        public_key=public_key,
        kind=kind,
        content=content,
        tags=[
            ['a', f'30009:{public_key}:{badge_d}'],
            ['p', f'{assign_to}', 'wss://relay.snort.social']
        ],
    )

In [14]:
# Create badge assignment event
assign_cake = assign_badge(public_key=privkey.public_key.hex(), assign_to=BOB, badge_d='cake-69')

print(assign_cake.id)

c9d48b8a5866456b26b331bcb369f0fc3ebfa2da0cd2fc19db6486c234c4508d


In [15]:
# Sign badge assignment event
privkey.sign_event(assign_cake)

In [16]:
# Publish badge assignment event
relay_manager.publish_event(assign_cake)

# ...
# Cake Card is Used
# ...

In [17]:
# Create badge consumption assignment event
assign_cake_consumed = assign_badge(public_key=privkey.public_key.hex(),
                                    assign_to=BOB, badge_d='poop-69')

print(assign_cake_consumed.id)

70799e62ba1a57bfe1ae34fc7b3b191f63bc12d3e5f7aafa0aef956d42fe80f2


In [18]:
# Sign badge consumption assignment event
privkey.sign_event(assign_cake_consumed)

In [19]:
# Publish badge consumption assignment event
relay_manager.publish_event(assign_cake_consumed)

In [2]:
RELAY: str = 'wss://nor.st'
ALICE: str = 'nsec1mp95lpaxpd50dvvh7cq0azasvdl8cd0qgu55qtlmz3he25wj2t2q52lx9k'
BOB: str = '4b9c064b4863cc0d02d794f4fff9e7793ddadd93175623987642fe073b5d5dd2'